## **Configure the model training on the Google AI Platform**

In [1]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Package_Path = 'ai_platform_train'
# !ls -l
# !mkdir {Package_Path}
# !touch {Package_Path}/__init__.py
# !ls -l {Package_Path}


Mounted at /content/drive/


In [2]:
## to the work directory
import os
work_dir = "/content/drive/My Drive/Earth-Engine-with-Deep-Learning/trainer/ai_platform_package"
os.chdir(work_dir)


### **Prepare the configuration file.**

In [3]:
%%writefile config.py

import tensorflow as tf

### Specific for the the Goolge Cloud Platform
######################################################################
# Insert the project id and the Bucket!
Project = 'my-project-20200813'
Bucket = 'earth-engine-bucket-1'
# Specify names of output locations in Cloud Storage.
Folder = 'ai_platform_train/unet_256_l8l7_50epoch'
Job_Dir = 'gs://' + Bucket + '/' + Folder
Model_Dir = Job_Dir + '/model'
Logs_Dir = Job_Dir + '/logs'
# Put the EEified model next to the trained model directory.
EEified_Dir = Job_Dir + '/eeified'
# training data folder and name
Image_Folder_tra = 'MSMT_RF_impervious_traData'   # !can't write into the second-level directory
Image_Folder_eva = 'MSMT_RF_impervious_evaData'

#######################################################################

## TFRecord features
# output bands
Bands_l8 = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
Bands_l57 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
Targets = ['impervious']
Features_l8 = Bands_l8 + Targets
Features_l57 = Bands_l57 + Targets

# Specify the size and shape of patches expected by the model.
Kernel_shape = [256, 256]
Columns_l8 = [
  tf.io.FixedLenFeature(shape=Kernel_shape, dtype=tf.float32) for k in Features_l8
]
Features_Dict_l8 = dict(zip(Features_l8, Columns_l8))

Columns_l57 = [
  tf.io.FixedLenFeature(shape=Kernel_shape, dtype=tf.float32) for k in Features_l57
]

Features_Dict_l57 = dict(zip(Features_l57, Columns_l57))

# Specify model training parameters.
Batch_Size = 16
Epochs = 50
Buffer_Size = 2000
Optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9)
Loss = 'MeanSquaredError'
Metrics = ['RootMeanSquaredError']

Overwriting config.py


### Data loader

In [4]:
## Data loader
%%writefile dataLoader.py

from . import config
import tensorflow as tf
import random

# Dataset loading functions

tra_pattern_l57 = 'gs://' + config.Bucket + '/' + config.Image_Folder_tra + '/' + 'Train_Landsat7*.tfrecord.gz'
tra_pattern_l8 = 'gs://' + config.Bucket + '/' + config.Image_Folder_tra + '/' + 'Train_Landsat8*.tfrecord.gz'
eva_pattern_l57 = 'gs://' + config.Bucket + '/' + config.Image_Folder_eva + '/' + 'Eva_Landsat7*.tfrecord.gz'
eva_pattern_l8 = 'gs://' + config.Bucket + '/' + config.Image_Folder_eva + '/' + 'Eva_Landsat8*.tfrecord.gz'
print(tra_pattern_l57)

# Dataset loading functions
def parse_tfrecord_l57(example_proto):
	return tf.io.parse_single_example(example_proto, config.Features_Dict_l57)
 
def to_tuple_l57(inputs):
    inputsList = [inputs.get(key) for key in config.Features_l57]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(config.Bands_l57)], stacked[:,:,len(config.Bands_l57):]

def parse_tfrecord_l8(example_proto):
	return tf.io.parse_single_example(example_proto, config.Features_Dict_l8)
 
def to_tuple_l8(inputs):
    inputsList = [inputs.get(key) for key in config.Features_l8]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(config.Bands_l8)], stacked[:,:,len(config.Bands_l8):]

def image_aug(image, truth, flip = True, rot = True):
    if flip == True:
        if tf.random.uniform(()) > 0.5:
            if random.randint(1,2) == 1:  ## horizontal or vertical mirroring
                image = tf.image.flip_left_right(image)
                truth = tf.image.flip_left_right(truth)
            else: 
                image = tf.image.flip_up_down(image)
                truth = tf.image.flip_up_down(truth)
    if rot == True:
        if tf.random.uniform(()) > 0.5: 
            degree = random.randint(1,3)
            image = tf.image.rot90(image, k=degree)
            truth = tf.image.rot90(truth, k=degree)
    return image, truth

def get_training_dataset():
    ## for landsat 5&7
    glob_l57 = tf.io.gfile.glob(tra_pattern_l57)
    dataset_l57 = tf.data.TFRecordDataset(glob_l57, compression_type='GZIP')    
    dataset_l57 = dataset_l57.map(parse_tfrecord_l57)
    dataset_l57 = dataset_l57.map(to_tuple_l57)
    ## for landsat 8
    glob_l8 = tf.io.gfile.glob(tra_pattern_l8)
    dataset_l8 = tf.data.TFRecordDataset(glob_l8, compression_type='GZIP')
    dataset_l8 = dataset_l8.map(parse_tfrecord_l8)
    dataset_l8 = dataset_l8.map(to_tuple_l8)
    ## combination
    combined_dataset = dataset_l57.concatenate(dataset_l8)
    combined_dataset = combined_dataset.map(image_aug)
    combined_dataset = combined_dataset.shuffle(config.Buffer_Size).batch(config.Batch_Size).repeat()
    return combined_dataset

def get_eval_dataset():
    ## for landsat 5&7
    glob_l57 = tf.io.gfile.glob(eva_pattern_l57)
    dataset_l57 = tf.data.TFRecordDataset(glob_l57, compression_type='GZIP')    
    dataset_l57 = dataset_l57.map(parse_tfrecord_l57)
    dataset_l57 = dataset_l57.map(to_tuple_l57)
    ## for landsat 8
    glob_l8 = tf.io.gfile.glob(eva_pattern_l8)
    dataset_l8 = tf.data.TFRecordDataset(glob_l8, compression_type='GZIP')
    dataset_l8 = dataset_l8.map(parse_tfrecord_l8)
    dataset_l8 = dataset_l8.map(to_tuple_l8)
    ## combination
    combined_dataset = dataset_l57.concatenate(dataset_l8)
    combined_dataset = combined_dataset.shuffle(config.Buffer_Size).batch(1).repeat()
    return combined_dataset

Overwriting dataLoader.py


## Model 

In [5]:
## model building
%%writefile model.py

import tensorflow as tf

############## U-Net
###  Define the downsample function
##   Conv2D+BN+ReLU
def downsample(filters, size, apply_dropout=True):
    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer='he_normal', use_bias=True))
    result.add(tf.keras.layers.BatchNormalization())
    result.add(tf.keras.layers.ReLU())

    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer='he_normal', use_bias=True))
    result.add(tf.keras.layers.BatchNormalization())
    result.add(tf.keras.layers.ReLU())
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.3))
    return result

### Define the upsample function
##  TransposeConv2D+BN+ReLU
def upsample(filters, size, apply_dropout=True):
    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                             kernel_initializer='he_normal',  use_bias=True))       
    result.add(tf.keras.layers.BatchNormalization())
    result.add(tf.keras.layers.ReLU())
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=1, padding='same',
                             kernel_initializer='he_normal', use_bias=True))
    result.add(tf.keras.layers.BatchNormalization())
    result.add(tf.keras.layers.ReLU())
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.3))
    return result

## Simple U-Net
def UNet(input_shape, nclasses=2):
    ## encoder of the U-Net
    (img_height, img_width, img_channel) = input_shape
    down_stack = [
        downsample(32, 3), # outp: (bs, img_height/2, img_width/2, 32)
        downsample(64, 3), # (bs, img_height/4, img_width/4, 64)
        downsample(64, 3), # (bs, img_height/8, img_width/8, 128)
        downsample(128, 3), # (bs, img_height/16, img_width/16, 256)
        downsample(128, 3), # (bs, img_height/32, img_width/32, 512)
        downsample(256, 3), # (bs, img_height/64, img_width/64, 512)
        # downsample(256, 3), # (bs, img_height/128, img_width/128, 512)
    ]

    ## decoder of the U-Net
    up_stack = [
        # upsample(256, 3), # output: (bs, img_height/64, img_width/64, 1024)
        upsample(256, 3), # (bs, img_height/32, img_width/32, 1024)
        upsample(128, 3), # (bs, img_height/16, img_width/16, 1024)
        upsample(64, 3), # (bs, img_height/8, img_width/8, 512)
        upsample(64, 3), # (bs, img_height/4, img_width/4, 256)
        upsample(32, 3), # (bs, img_height/2, img_width/2, 128)
    ]

    # define the input and output tensors
    inputs = tf.keras.layers.Input(shape=[img_height, img_width, img_channel])
    
    if nclasses == 2:        
        last = tf.keras.layers.Conv2D(1, 1, strides=1, padding='same',
                    kernel_initializer='he_normal', activation= 'sigmoid')  ## (bs, 256, 256, 1)
    else:
        last = tf.keras.layers.Conv2D(nclasses, 1, strides=1, padding='same',
                    kernel_initializer='he_normal', activation= 'softmax')  ## (bs, 256, 256, 1)

    concat = tf.keras.layers.Concatenate()    
    x = inputs
    # Downsampling through the model
    skips = []   # reserve the output of medium output of the encoder network 
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])  #  
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = concat([x, skip])
    x = upsample(32, 3)(x)
    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


Overwriting model.py


In [6]:
# from model import UNet
# model = UNet((256,256,6), nclasses=2)
# model.summary()

## Check the dataloader functions and the built model

## Training task on AI Platform

In [7]:
%%writefile trainingTask.py

from . import config
from . import model
from . import dataLoader
import tensorflow as tf

if __name__ == '__main__':

    training = dataLoader.get_training_dataset()
    evaluation = dataLoader.get_eval_dataset()

    model = model.UNet(input_shape=(config.Kernel_shape[0], config.Kernel_shape[1], 6), nclasses=2)

    model.compile(
		optimizer=tf.keras.optimizers.get(config.Optimizer),
		loss=tf.keras.losses.get(config.Loss),
		metrics=[tf.keras.metrics.get(metric) for metric in config.Metrics])

    model.fit(
        x=training,
        epochs=config.Epochs, 
        steps_per_epoch=int(1000*6/16),
        validation_data=evaluation,
        validation_steps=300*6,
        callbacks=[tf.keras.callbacks.TensorBoard(config.Logs_Dir)]
        )

    model.save(config.Model_Dir, save_format='tf')


Overwriting trainingTask.py
